## Bedrcok Hello World

In [1]:
import boto3, json
from botocore.exceptions import ClientError

session = boto3.Session(profile_name="bedrock-dev")
client = session.client("bedrock-runtime", region_name="us-east-1")
model_id = "amazon.nova-lite-v1:0"
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




The purpose of a 'hello world' program is to provide a simple introduction to the syntax and basic structure of a programming language.


In [ ]:
### For Chroma

import os
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

load_dotenv()
client = OpenAI()

CHAT_MODEL = "gpt-4o-mini"
EMB_MODEL  = "text-embedding-3-small"

CORPUS = [
    ("doc1", """Provectus is an AI/ML company focusing on GenAI, RAG, and agentic systems.
They specialize in AWS Bedrock, OpenSearch, and reproducible ML pipelines."""),
    ("doc2", """The ML Engineer role requires experience with LLMs in production, RAG, agentic systems,
classification/regression tasks, and solid Python + Docker skills."""),
    ("doc3", """Provectus is located in Austin, TX, and has offices in the US and Ukraine."""),
]


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts, metas = [], []
for doc_id, text in CORPUS:
    for ch in splitter.split_text(text):
        texts.append(ch) 
        metas.append({"source": doc_id})


emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, emb, metadatas=metas)
retriever = vs.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)
res = qa.invoke({"query": "Where is provectus located?"})
print(res["result"])


Provectus is located in Austin, TX, and has offices in the US and Ukraine.
Sources: {'doc2', 'doc3', 'doc1'}


LangChain RAg

In [3]:
import os
from typing import List, Dict
import logging
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
logging.basicConfig(level=logging.INFO)

class RAGSystem:
    def __init__(self, model_name="gpt-3.5-turbo", embedding_model="all-MiniLM-L6-v2"):
        self.llm = ChatOpenAI(model=model_name, temperature=0)
        self.embedding = HuggingFaceEmbeddings(model_name=embedding_model)
        self.vectorstore = None
        self.retriever = None
        self.qa_chain = None

    def load_and_chunk_text(self, text: str, chunk_size=500, chunk_overlap=50):
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        return splitter.split_text(text)

    def build_index(self, texts: List[str]):
        self.vectorstore = FAISS.from_texts(texts, self.embedding)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 3})

    def setup_chain(self):
        template = """Answer the question based only on the following context:
        {context}

        Question: {question}
        If you don't know, say "I don't know".
        Answer:"""
        prompt = ChatPromptTemplate.from_template(template)
        self.qa_chain = (
            {"context": self.retriever, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

    def query(self, question: str) -> Dict:
        try:
            if not self.qa_chain:
                raise ValueError("Chain not initialized. Call setup_chain() first.")
            answer = self.qa_chain.invoke(question)
            # Get sources (optional)
            docs = self.retriever.invoke(question)
            sources = [doc.page_content[:100] + "..." for doc in docs]
            return {
                "answer": answer,
                "sources": sources,
                "confidence": "high" if len(answer) > 10 else "low"
            }
        except Exception as e:
            logging.error(f"Query failed: {e}")
            return {"answer": "Error occurred", "sources": [], "confidence": "none"}

# === USAGE ===
if __name__ == "__main__":
    # Sample text
    sample_text = """
    Provectus is an AI and ML consulting company focused on building cutting-edge solutions.
    They specialize in GenAI, RAG, agentic systems, and cloud-based ML pipelines.
    Offices are located in SA, Canada, and Mozambique.
    """

    rag = RAGSystem()
    chunks = rag.load_and_chunk_text(sample_text)
    rag.build_index(chunks)
    rag.setup_chain()

    result = rag.query("Where is Provectus located?")
    print(result)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'answer': 'SA, Canada, and Mozambique', 'sources': ['Provectus is an AI and ML consulting company focused on building cutting-edge solutions.\n    They sp...'], 'confidence': 'high'}


Tool Calling

In [11]:
# agent_template.py
import os
import re
import json
from typing import Dict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

load_dotenv()

@tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression. Use for +, -, *, /, **."""
    try:
        # Safe eval for demo (in prod, use ast.literal_eval or numexpr)
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

@tool
def count_letters(text: str) -> dict:
    """Count letters from text and return a dictionary {letter: count}."""
    counts = {}
    for char in text:
        if char.isalpha():  # Only count letters
            char = char.lower()
            counts[char] = counts.get(char, 0) + 1
    return counts

tools = [calculator, count_letters]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with tools."),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

def run_agent(query: str) -> Dict:
    try:
        response = agent_executor.invoke({"input": query})
        return {
            "output": response["output"],
            "steps": len(response.get("intermediate_steps", [])),
            "status": "success"
        }
    except Exception as e:
        return {"output": f"Agent failed: {e}", "steps": 0, "status": "error"}

# === USAGE ===
if __name__ == "__main__":
    result = run_agent("how many r's are in strawberry?")
    print(result)



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'text': 'strawberry'}`


{'s': 1, 't': 1, 'r': 3, 'a': 1, 'w': 1, 'b': 1, 'e': 1, 'y': 1}There are 3 'r's in the word "strawberry".

> Finished chain.
{'output': 'There are 3 \'r\'s in the word "strawberry".', 'steps': 0, 'status': 'success'}


In [ ]:
# llm_wrapper_template.py
import os
import time
import logging
from typing import Dict, Optional
from tenacity import retry, stop_after_attempt, wait_exponential
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
logging.basicConfig(level=logging.INFO)

class LLMWrapper:
    def __init__(self, model="gpt-3.5-turbo", max_retries=3):
        self.llm = ChatOpenAI(model=model, temperature=0.7)
        self.max_retries = max_retries
        self.metrics = []

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
    def _safe_invoke(self, prompt: str):
        return self.llm.invoke(prompt)

    def generate(self, prompt: str, task_id: str = "default") -> Dict:
        start = time.time()
        try:
            response = self._safe_invoke(prompt)
            latency = time.time() - start
            tokens_in = len(prompt.split())  # rough approx
            cost = tokens_in * 0.000001  # placeholder

            result = {
                "output": response.content,
                "latency": round(latency, 2),
                "tokens_in": tokens_in,
                "cost": round(cost, 6),
                "status": "success",
                "task_id": task_id
            }
            self.metrics.append(result)
            return result

        except Exception as e:
            latency = time.time() - start
            result = {
                "output": "",
                "latency": round(latency, 2),
                "tokens_in": len(prompt.split()),
                "cost": 0,
                "status": f"error: {str(e)}",
                "task_id": task_id
            }
            self.metrics.append(result)
            logging.error(f"LLM call failed: {e}")
            return result

    def get_metrics(self):
        return self.metrics

# === USAGE ===
if __name__ == "__main__":
    wrapper = LLMWrapper()
    result = wrapper.generate("Explain RAG in one sentence.", "task_1")
    print(result)
    print("Metrics:", wrapper.get_metrics())

c:\Users\kevor\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


AttributeError: type object 'SentimentResponse' has no attribute 'model_json_schema'

In [ ]:
# bedrock_template.py
import json
import boto3
import logging
from typing import Dict

logging.basicConfig(level=logging.INFO)

class BedrockClient:
    def __init__(self, region="us-east-1", model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
        self.client = boto3.client("bedrock-runtime", region_name=region)
        self.model_id = model_id

    def invoke(self, prompt: str, max_tokens=500, temperature=0.7) -> Dict:
        try:
            # Claude 3 format
            body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "temperature": temperature,
                "messages": [
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": prompt}]
                    }
                ]
            })

            response = self.client.invoke_model(
                modelId=self.model_id,
                body=body
            )

            response_body = json.loads(response["body"].read())
            output = response_body["content"][0]["text"]

            return {
                "output": output,
                "status": "success",
                "model": self.model_id
            }

        except Exception as e:
            logging.error(f"Bedrock error: {e}")
            return {"output": "", "status": f"error: {str(e)}", "model": self.model_id}

# === USAGE ===
if __name__ == "__main__":
    # ⚠️ Ensure AWS credentials are set in ~/.aws/credentials or env vars
    bedrock = BedrockClient()
    result = bedrock.invoke("Explain what RAG is in one sentence.")
    print(result)

In [ ]:
# eval_template.py
import re
from typing import Dict

class LLMEvaluator:
    @staticmethod
    def keyword_match(output: str, expected_keywords: list) -> Dict:
        found = [kw for kw in expected_keywords if kw.lower() in output.lower()]
        score = len(found) / len(expected_keywords) if expected_keywords else 0
        return {"metric": "keyword_recall", "score": score, "found": found}

    @staticmethod
    def contains_forbidden(output: str, forbidden: list) -> bool:
        return any(f.lower() in output.lower() for f in forbidden)

    @staticmethod
    def answer_length(output: str) -> Dict:
        word_count = len(output.split())
        return {
            "metric": "length",
            "word_count": word_count,
            "category": "short" if word_count < 10 else "medium" if word_count < 50 else "long"
        }

    @staticmethod
    def run_all(output: str, expected_keywords=[], forbidden=[]) -> Dict:
        return {
            "keyword_eval": LLMEvaluator.keyword_match(output, expected_keywords),
            "forbidden_found": LLMEvaluator.contains_forbidden(output, forbidden),
            "length_eval": LLMEvaluator.answer_length(output)
        }

# === USAGE ===
if __name__ == "__main__":
    output = "RAG stands for Retrieval-Augmented Generation. It combines retrieval and generation."
    eval_result = LLMEvaluator.run_all(
        output,
        expected_keywords=["retrieval", "generation", "RAG"],
        forbidden=["I don't know", "sorry"]
    )
    print(eval_result)

In [ ]:
# experiment_template.py
import json
import time
from datetime import datetime
from typing import List, Dict

class ExperimentTracker:
    def __init__(self, experiment_name: str):
        self.experiment_name = experiment_name
        self.runs = []
        self.start_time = time.time()

    def log_run(self, params: Dict, result: Dict, notes: str = ""):
        run = {
            "timestamp": datetime.now().isoformat(),
            "params": params,
            "result": result,
            "notes": notes,
            "duration_sec": time.time() - self.start_time
        }
        self.runs.append(run)
        print(f"[Experiment] Logged run with params: {params}")

    def save_to_file(self, filename: str = None):
        if not filename:
            filename = f"{self.experiment_name}_{int(time.time())}.json"
        with open(filename, 'w') as f:
            json.dump(self.runs, f, indent=2)
        print(f"Saved {len(self.runs)} runs to {filename}")

    def get_summary(self):
        successes = sum(1 for r in self.runs if r["result"].get("status") == "success")
        total = len(self.runs)
        avg_latency = sum(r["result"].get("latency", 0) for r in self.runs) / total if total > 0 else 0
        return {
            "total_runs": total,
            "success_rate": successes / total if total > 0 else 0,
            "avg_latency": round(avg_latency, 2)
        }

# === USAGE ===
if __name__ == "__main__":
    tracker = ExperimentTracker("rag_comparison_v1")

    # Simulate 2 runs
    tracker.log_run(
        params={"model": "gpt-3.5", "chunk_size": 500},
        result={"answer": "Yerevan", "latency": 2.1, "status": "success"},
        notes="Baseline RAG"
    )

    tracker.log_run(
        params={"model": "claude-sonnet", "chunk_size": 300},
        result={"answer": "Belgrade", "latency": 3.5, "status": "success"},
        notes="Smaller chunks"
    )

    print(tracker.get_summary())
    tracker.save_to_file()

In [12]:
import os, argparse, glob
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA


In [2]:
sample_text = """
    Provectus is an AI and ML consulting company focused on building cutting-edge solutions.
    They specialize in GenAI, RAG, agentic systems, and cloud-based ML pipelines.
    Offices are located in SA, Canada, and Mozambique.
    """


: 